In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import string
import math
import re

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16584205962796674014
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4815519744
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16393774557564032374
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
SEQUENCE_LEN = 60
BATCH_SIZE = 512
EPOCHS = 20
LSTM_DIM = 256
LAYER_COUNT = 4
DROPOUT = 0.2
TEXT_CLEANING_RE = "[\t\n\r_`]"

In [5]:
with open("./data/complete_lotr_precleaned.txt", "rb") as f:
    text = f.read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 2529564 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])


This tale grew in the telling, until it became a history of the Great War of the Ring and included many glimpses of the yet more ancient history that preceded it. It was begun soon after _The Hobbit_ was written and before its publication in 1937; 


In [7]:
#Generating Vocabulary, character to index  and index to character dictionary
text = re.sub(" +"," ",text.lower())
text = re.sub(TEXT_CLEANING_RE,"", text)

vocab = sorted(set(text))
vocab_size = len(vocab)
char_to_idx = {c:i for i,c in enumerate(vocab)}
idx_to_char = {i:c for c,i in char_to_idx.items()}

print('{} unique characters'.format(len(vocab)))
print(char_to_idx.items())

63 unique characters
dict_items([(' ', 0), ('!', 1), ('"', 2), ("'", 3), ('(', 4), (')', 5), ('*', 6), (',', 7), ('-', 8), ('.', 9), ('/', 10), ('0', 11), ('1', 12), ('2', 13), ('3', 14), ('4', 15), ('5', 16), ('6', 17), ('7', 18), ('8', 19), ('9', 20), (':', 21), (';', 22), ('=', 23), ('?', 24), ('a', 25), ('b', 26), ('c', 27), ('d', 28), ('e', 29), ('f', 30), ('g', 31), ('h', 32), ('i', 33), ('j', 34), ('k', 35), ('l', 36), ('m', 37), ('n', 38), ('o', 39), ('p', 40), ('q', 41), ('r', 42), ('s', 43), ('t', 44), ('u', 45), ('v', 46), ('w', 47), ('x', 48), ('y', 49), ('z', 50), ('á', 51), ('â', 52), ('ä', 53), ('é', 54), ('ê', 55), ('ë', 56), ('í', 57), ('î', 58), ('ó', 59), ('ú', 60), ('û', 61), ('–', 62)])


In [8]:
def batch_generator(text, batch_size = BATCH_SIZE, sequence_size = SEQUENCE_LEN, vocab_size = vocab_size):
    n_steps = (len(text)-sequence_size) // (batch_size)
    while True:
        for batch_idx in range(n_steps):
            x = np.zeros((batch_size,sequence_size,vocab_size))
            y = np.zeros((batch_size, vocab_size))

            batch_start = batch_idx*batch_size

            for sents_idx in range(batch_size):
              sent_start = batch_start + sents_idx

              for s in range(sequence_size):
                x[sents_idx, s, char_to_idx[text[sent_start + s]]] = 1
              y[sents_idx,char_to_idx[text[sent_start + s + 1]]] = 1
            
            yield x,y 

In [9]:
def build_model():
    model = tf.keras.models.Sequential()
    for i in range(LAYER_COUNT):
        model.add(
            tf.keras.layers.LSTM(LSTM_DIM,
                                 return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                                 input_shape = (SEQUENCE_LEN, vocab_size)
                                )
            )
        model.add(tf.keras.layers.Dropout(DROPOUT))
    model.add(tf.keras.layers.Dense(vocab_size,
                                    activation='softmax'
                                   ))
    adam = tf.keras.optimizers.Adam()
    
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

    

In [10]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                             min_delta=0, patience=3,
                                             verbose=0,
                                             mode='auto')
filepath = "model_weights_saved.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,
                                                monitor='loss', verbose=1, save_best_only=True, 
                                                mode='min')
desired_callbacks = [earlystop, checkpoint]
training_model = build_model()    
print(training_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 256)           327680    
_________________________________________________________________
dropout (Dropout)            (None, 60, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 256)           525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               5

In [11]:
history = training_model.fit_generator(
    batch_generator(text),
    steps_per_epoch = (len(text)-SEQUENCE_LEN) // (BATCH_SIZE),
    #max_queue_size=1, # no more than one queued batch in RAM
    epochs=EPOCHS,
    callbacks=desired_callbacks,
    initial_epoch=0
)

Epoch 1/20


C:\Users\utsav\anaconda3\envs\NLP-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


4835/4835 [==============================] - 692s 142ms/step - loss: 2.3587 - accuracy: 0.3251

Epoch 00001: loss improved from inf to 1.97432, saving model to model_weights_saved.hdf5
Epoch 2/20
4835/4835 [==============================] - 712s 147ms/step - loss: 2.1740 - accuracy: 0.3682

Epoch 00002: loss did not improve from 1.97432
Epoch 3/20
4835/4835 [==============================] - 715s 148ms/step - loss: 1.6906 - accuracy: 0.4916

Epoch 00003: loss improved from 1.97432 to 1.63427, saving model to model_weights_saved.hdf5
Epoch 4/20
4835/4835 [==============================] - 719s 149ms/step - loss: 1.6487 - accuracy: 0.5025

Epoch 00004: loss improved from 1.63427 to 1.60440, saving model to model_weights_saved.hdf5
Epoch 5/20
4835/4835 [==============================] - 719s 149ms/step - loss: 1.4860 - accuracy: 0.5459

Epoch 00005: loss improved from 1.60440 to 1.44142, saving model to model_weights_saved.hdf5
Epoch 6/20
4835/4835 [==============================] - 720s 

In [12]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
plot_graphs(history, 'loss')